> https://geckopy.readthedocs.io/en/latest/01-getting-started.html

> https://github.com/ginkgobioworks/geckopy (最新)

- geckopyはPyPIには登録されていない

- CPLEXの環境構築については、notion>環境構築を参照する
　(仮想環境 "gecko" に入った後に、bashで `docplex config --upgrade "/opt/ibm/ILOG/CPLEX_Studio2212"`)

In [1]:
import numpy as np
import pandas as pd
import cobra
from cobra.io import load_matlab_model
from cobra.io import write_sbml_model
from cobra.flux_analysis.variability import flux_variability_analysis
import geckopy
from geckopy.io import sbml

### Import models

In [2]:
# パス定義
yeast_ecgem_mat = r"/home/daiki/Github/ENEOS/python/models/ecYeastGEM_batch (CellFactory-ecYeastGEM).mat"
yeast_ecgem_xml = r"/home/daiki/Github/ENEOS/python/models/ecYeastGEM_batch (CellFactory-ecYeastGEM).xml"

In [ ]:
# matを読み込み
mat_model = load_matlab_model(yeast_ecgem_mat)

# SBMLで保存
write_sbml_model(mat_model, yeast_ecgem_xml)

In [3]:
# geckopyでSBMLを読み込み
ec_model = sbml.read_sbml_ec_model(yeast_ecgem_xml)

### Python API: Protein isolation

In [4]:
solution_rxn, solution_prot = ec_model.optimize() # cobrapyの solution オブジェクトを返す

# ec_model.slim_optimize() は目的関数の値（成長率など）を返すだけなので高速

In [5]:
solution_rxn # 各反応のフラックス (.fluxes)

,fluxes,reduced_costs
r_0006,20.622659,0.000000
r_0070,0.000000,0.000000
r_0094,0.000000,-0.096531
r_0099,0.000000,0.000000
r_0200,0.000000,0.000000
...,...,...
r_4484_REVNo1,0.000000,0.000000
r_4484_REVNo2,0.000000,0.000000
r_4484_REVNo3,0.000000,0.000000
draw_prot_A2P2R3,0.000000,0.000000


In [6]:
solution_prot # タンパク質濃度とタンパク質影響度 (.contribution)(双対変数)

,fluxes,reduced_costs
prot_P00045,0.0,0.0
prot_P32891,0.0,0.0
prot_P00044,0.0,0.0
prot_P39976,0.0,0.0
prot_P46681,0.0,0.0
...,...,...
prot_Q04119,0.0,0.0
prot_P38995,0.0,0.0
prot_P49954,0.0,0.0
prot_P39979,0.0,0.0


### Kcats

- geckopy では、各酵素-反応ペアに対応する触媒定数 ($k_{cat}$) を `Protein`オブジェクト経由で **辞書型 (dict)** として扱う。
- $k_{cat}$ の入力単位はユーザー側が指定するのは $1/s$ $(秒^{-1})$ だが、内部では $1/h$ $(時間^{-1})$ に変換されている。
$$
k_{cat} [s^{-1}] \times 3600 \to k_{cat} [h^{-1}]
$$

In [7]:
# kcat の参照と操作
print(ec_model.proteins.prot_P00045.kcats) # Proteinオブジェクト(prot_P00045)のkcatを確認
print(ec_model.proteins.prot_P00045.kcats["r_0004No1"]) # "r_0004No1" に対応する kcat

ec_model.proteins.prot_P00045.kcats["r_0004No1"] = 100 # 新たに kcat を設定

{<Reaction r_0001No1 at 0x7fd10558ba00>: 1499.9610010139734, <Reaction r_0437No1 at 0x7fd10528a610>: 999.9920000639995, <Reaction r_0002No1 at 0x7fd10558ba30>: 1499.9610010139734, <Reaction r_0001No3 at 0x7fd10558be80>: 1499.9610010139734, <Reaction r_0004No1 at 0x7fd105593bb0>: 372.0022201092496}
372.0022201092496


In [8]:
# 化学量論行列 S の量論係数 s を取得
ec_model.reactions.r_0004No1.metabolites[ec_model.proteins.prot_P00045] # - 1/kcat (実際は - 1/(kcat*3600) の値が出力される) 

-2.777777777777778e-06

In [9]:
ec_model.proteins.prot_P00045.kcats["r_0004No1"] = 120 # 元の値が60に対して2倍の kcat を設定すれば、s は 1/2倍される
ec_model.reactions.r_0004No1.metabolites[ec_model.proteins.prot_P00045]

-2.3148148148148148e-06

## Thermodynamics integretion

- geckopyは、pytfaを通してTFA (thermodynamics flux analysis) と enzyme constraint model を統合できる。

---

### Load the model

### Plain FVA

In [7]:
ecM = ec_model.copy()
ecM.solver = 'cplex'
print(ecM.objective)

#ecM.objective = "r_4041" # biomass reaction
#print(ecM.objective)
print(ecM.medium)

Maximize
1.0*r_2111 - 1.0*r_2111_reverse_58b69
{'r_1654_REV': inf, 'r_1714_REV': 1000.0, 'r_1832_REV': inf, 'r_1861_REV': inf, 'r_1992_REV': inf, 'r_2005_REV': inf, 'r_2020_REV': inf, 'r_2049_REV': inf, 'r_2060_REV': inf, 'r_2100_REV': inf, 'r_4593_REV': inf, 'r_4594_REV': inf, 'r_4595_REV': inf, 'r_4596_REV': inf, 'r_4597_REV': inf, 'r_4600_REV': inf}


In [8]:
for rxn_id in ecM.medium.keys():
    rxn = ecM.reactions.get_by_id(rxn_id)
    print(rxn.id, rxn.name, rxn.bounds)

r_1654_REV ammonium exchange (reversible) (0.0, inf)
r_1714_REV D-glucose exchange (reversible) (0.0, 1000.0)
r_1832_REV H+ exchange (reversible) (0.0, inf)
r_1861_REV iron(2+) exchange (reversible) (0.0, inf)
r_1992_REV oxygen exchange (reversible) (0.0, inf)
r_2005_REV phosphate exchange (reversible) (0.0, inf)
r_2020_REV potassium exchange (reversible) (0.0, inf)
r_2049_REV sodium exchange (reversible) (0.0, inf)
r_2060_REV sulphate exchange (reversible) (0.0, inf)
r_2100_REV water exchange (reversible) (0.0, inf)
r_4593_REV chloride exchange (reversible) (0.0, inf)
r_4594_REV Cu2(+) exchange (reversible) (0.0, inf)
r_4595_REV Mn(2+) exchange (reversible) (0.0, inf)
r_4596_REV Zn(2+) exchange (reversible) (0.0, inf)
r_4597_REV Mg(2+) exchange (reversible) (0.0, inf)
r_4600_REV Ca(2+) exchange (reversible) (0.0, inf)


---
### 通常のモデルとexModelの交換反応の違い

#### 通常のExchange反応
- 例えば、 `EX_glc__D_e` のように名前がついたものは、モデル上では **一つの反応** として定義されており、
    - `lower_bound < 0` を許すと「取り込み (uptake)」
    - `upper_bound > 0` を許すと「排出 (secretion)」


    というように、 **双方向** を同じ反応で表現する。

#### "_REV"付きのExchange反応
- 一方で、exGEMなどの一部のモデルでは、双方向を **別々の反応** に分解している。
 1. `EX_glc__D_e` (非"_REV") : 主に「排出 (生産)」方向を表す
 2. `EX_glc__D_e_REV` : 主に「取り込み (消費)」方向を表す
- このように、反応ごとに上限・下限を独立に設定できるため、制御がやりやすくなっている。
---

In [9]:
# すべての可逆型取り込み反応 (uptake) をいったん閉じる
for ex_rxn in ecM.exchanges:
    if "_REV" in ex_rxn.id:
        ex_rxn.bounds = 0.0, 0.0

In [10]:
# 各種排出反応を調整
ecM.reactions.r_1992.upper_bound = 0.0 # oxygen
ecM.reactions.r_1714.upper_bound = 0.0 # glucose
ecM.reactions.r_1718.upper_bound = 0.0 # xylose
ecM.reactions.r_1761.upper_bound = 1000.0 # ethanol
#ecM.reactions.r_1709.upper_bound = 0.0 # fructose
#ecM.reactions.r_1715.upper_bound = 0.0 # mannose

In [11]:
ecM.reactions.r_2111.bounds = 0.0, 1000.0 # biomass reacions

ecM.reactions.r_1714_REV.bounds = 0.0, 10.0 # glucose exchange
ecM.reactions.r_1718_REV.bounds = 0.0, 10.0 # xylose exchange
ecM.reactions.r_1093No1.bounds = 0.0, 1000.0 # xylose reductase
ecM.reactions.r_2104_REV.bounds = 0.0, 0.0 # xylitol exchange
ecM.reactions.r_1092No1.bounds = 0.0, 1000.0 # xylitol dehydrogenase
ecM.reactions.r_4490.bounds = 0.0, 1000.0 # xylulose⇒xylitol
ecM.reactions.r_1654_REV.bounds = 0.0, 1000.0 # ammonium exchange
ecM.reactions.r_1992_REV.bounds = 0.0, 10.0 # oxygen exchange
ecM.reactions.r_1832_REV.bounds = 0.0, 1000.0 # proton exchange
ecM.reactions.r_1861_REV.bounds = 0.0, 1000.0 # iron(2+) exchange
ecM.reactions.r_2005_REV.bounds = 0.0, 1000.0 # phosphate exchange
ecM.reactions.r_2020_REV.bounds = 0.0, 1000.0 # potassium exchange
ecM.reactions.r_2049_REV.bounds = 0.0, 1000.0 # sodium exchange
ecM.reactions.r_2060_REV.bounds = 0.0, 1000.0 # sulphate exchange
ecM.reactions.r_2100_REV.bounds = 0.0, 1000.0 # water exchange
ecM.reactions.r_4593_REV.bounds = 0.0, 1000.0 # chloride change
ecM.reactions.r_4594_REV.bounds = 0.0, 1000.0 # Cu2(+) exchange
ecM.reactions.r_4595_REV.bounds = 0.0, 1000.0 # Mn(2+) exchange
ecM.reactions.r_4596_REV.bounds = 0.0, 1000.0 # Zn(2+) exchange
ecM.reactions.r_4597_REV.bounds = 0.0, 1000.0 # Mg(2+) exchange
ecM.reactions.r_4600_REV.bounds = 0.0, 1000.0 # Ca(2+) exchange

In [12]:
ecM.medium

{'r_1654_REV': 1000.0,
 'r_1714_REV': 10.0,
 'r_1718_REV': 10.0,
 'r_1832_REV': 1000.0,
 'r_1861_REV': 1000.0,
 'r_1992_REV': 10.0,
 'r_2005_REV': 1000.0,
 'r_2020_REV': 1000.0,
 'r_2049_REV': 1000.0,
 'r_2060_REV': 1000.0,
 'r_2100_REV': 1000.0,
 'r_4593_REV': 1000.0,
 'r_4594_REV': 1000.0,
 'r_4595_REV': 1000.0,
 'r_4596_REV': 1000.0,
 'r_4597_REV': 1000.0,
 'r_4600_REV': 1000.0}

In [13]:
ecM.optimize()

(<Solution 0.652 at 0x7f39b9cf46d0>, <Solution 0.652 at 0x7f399bd595e0>)

In [14]:
ecM.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
s_0420,r_1654_REV,4.465,0,0.00%
s_0565,r_1714_REV,10,6,54.55%
s_0579,r_1718_REV,10,5,45.45%
s_0796,r_1832_REV,0.1817,0,0.00%
s_0925,r_1861_REV,2.047E-05,0,0.00%
s_1277,r_1992_REV,10,0,0.00%
s_1324,r_2005_REV,0.1782,0,0.00%
s_1374,r_2020_REV,0.002366,0,0.00%
s_1438,r_2049_REV,0.002588,0,0.00%
s_1468,r_2060_REV,0.05967,0,0.00%


In [ ]:
ecM_dash = ecM.copy()
ecM_dash.reactions.r_1718_REV.bounds = 0.0, 0.0 # xylose exchange
ori_model_fva_fluxes = flux_variability_analysis(
    model = ecM_dash, 
    reaction_list = None,
    loopless = False, 
    fraction_of_optimum = 0.99, 
    pfba_factor = 1.1,
    processes = 8
)

ecM_fva_fluxes = flux_variability_analysis(
    model = ecM, 
    reaction_list = None,
    loopless = False, 
    fraction_of_optimum = 0.99, 
    pfba_factor = 1.1,
    processes = 8
)

In [31]:
print(ori_model_fva_fluxes.loc[["r_1761"]])
print(ecM_fva_fluxes.loc[["r_1761"]])

        minimum   maximum
r_1761  5.61104  7.853322
          minimum    maximum
r_1761  21.859459  24.160066


In [21]:
import plotly.express as px
dfs = (ori_model_fva_fluxes, ecM_fva_fluxes)
df_plot = pd.concat([abs(df.maximum - df.minimum).apply(lambda x: x if x < 2000 else 2000) for df in dfs], axis=1).melt(var_name="Variability method", value_name="Flux")
df_plot.loc[df_plot["Variability method"] == 0, "Variability method"] = "xylose -"
df_plot.loc[df_plot["Variability method"] == 1, "Variability method"] = "xylose +"
fig = px.histogram(
    df_plot,
    x="Flux", color="Variability method",
    cumulative=True, nbins=200,
    color_discrete_sequence=["rgba(26, 200, 26, 0.8)", "rgba(200, 26, 80, 0.5)"],
    marginal="violin", barmode="overlay",
)
fig.show(renderer="notebook_connected")